<a href="https://colab.research.google.com/github/JSJeong-me/GPT-DB/blob/main/Llamaindex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## LlamaIndex <> Langchain Integrations

This demo notebook shows how you can provide integrations between LlamaIndex and Langchain. It provides the following examples:
- Using LlamaIndex as a callable tool with a Langchain agent
- Using LlamaIndex as a memory module; this allows you to insert arbitrary amounts of conversation history with a Langchain chatbot!

In [1]:
!pip install --upgrade openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.4/225.4 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.1 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
tensorflow-probability 0.22.0 requires typing-extensions<4.6.0, but you have typing-extensions 4.9.0 which is incompatible.


In [2]:
!pip install llama-index pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.8/15.8 MB 66.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.0/143.0 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 72.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.3 MB/s eta 0:00:00
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.11.2
    Uninstalling beautifulsoup4-4.11.2:
      Successfully uninstalled beautifulsoup4-4.11.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.


In [3]:
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.3/803.3 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 205.3/205.3 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 7.0 MB/s eta 0:00:00


In [4]:
import openai

# Set up the OpenAI API client
openai.api_key = "sk-"


In [5]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

#### Using LlamaIndex as a Callable Tool

In [6]:
from langchain.agents import Tool
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent

from llama_index import VectorStoreIndex, SimpleDirectoryReader

In [7]:
!mkdir data

In [8]:
documents = SimpleDirectoryReader("./data").load_data()
index = VectorStoreIndex.from_documents(documents=documents)

In [9]:
tools = [
    Tool(
        name="LlamaIndex",
        func=lambda q: str(index.as_query_engine().query(q)),
        description="useful for when you want to answer questions about an insurance terms. The input to this tool should be a complete korean sentence.",
        return_direct=True,
    ),
]

In [10]:
# set Logging to DEBUG for more detailed outputs
memory = ConversationBufferMemory(memory_key="chat_history")
llm = ChatOpenAI(temperature=0, openai_api_key='sk-')
agent_executor = initialize_agent(
    tools, llm, agent="conversational-react-description", memory=memory
)

In [11]:
agent_executor.run(input="재해사망특약 상품코드는?")

'The product code for the accidental death rider is not provided in the given context information.'

In [12]:
agent_executor.run(input="납입완료보너스적립액에 대해 말해줘")

'The term "납입완료보너스적립액" refers to the amount that the company separately reserves in order to provide the "납입완료보너스" service. It is the amount of completed bonus that the company has set aside to provide the completed bonus service on the designated date.'

In [13]:
agent_executor.run(input="가산 후 납입완료보너스적립액에 대해 말해줘")

'가산 후 납입완료보너스적립액은 보험 가입자가 보험료를 납입한 후에 추가로 적립되는 보너스입니다. 이는 일반적으로 보험사가 보험 가입자에게 제공하는 혜택 중 하나로, 가입자의 보험료 납입 기간에 따라 적립액이 증가합니다. 가산 후 납입완료보너스적립액은 가입자가 보험 계약을 유지하고 보험료를 지속적으로 납입하는 동안 계속해서 적립됩니다. 이 적립액은 일정한 이자율에 따라 증가하며, 보험 계약 종료 시에는 가입자에게 지급되거나 다른 혜택으로 활용될 수 있습니다.'

In [14]:
agent_executor.run(input="주계약 보험료 납입완료시점이 10년이면 납입완료보너스 지급률은 몇 %이지?")

'The payment rate for the completion bonus is 24.1% if the payment completion time for the main contract premium is 10 years.'

In [17]:
agent_executor.run(input="보험료 납입면제에 대해 말해줘")

ValueError: ignored

In [18]:
agent_executor.run(input="중대사유로 인한 해지에 대해 알려줘")

ValueError: ignored

In [19]:
agent_executor.run(input="사망보험금의 연금선지급 서비스에서 체증형이 무엇이지?")

'체증형은 연금선지급금이 지급기간 동안 매년 감액되는 형태입니다. 단, 선지급금은 매년 변동할 수 있습니다.'

In [20]:
agent_executor.run(input="장해란 무었인가?")

'장해란 보험 계약자가 사고로 인해 일시적 또는 영구적으로 신체적인 손상을 입은 상태를 말합니다. 이 손상은 일상 생활이나 직업 활동에 제한을 줄 수 있으며, 장해 보험은 이러한 상황에서 보험금을 제공하는 보험 상품입니다. 장해 보험은 다양한 종류와 수준의 장해에 대해 보상을 제공할 수 있으며, 일반적으로 장해의 정도에 따라 보험금이 결정됩니다.'

In [21]:
agent_executor.run(input="두 눈이 멀었을 때 지급률은 몇 %이지?")


'100'

In [ ]:
agent_executor.run(input="코의 장해판정기준에 대해 설명해줘")


In [ ]:
agent_executor.run(input="코의 장해판정기준에 대해 설명해줘", handle_parsing_errors=True)

In [ ]:
agent_executor.run(input="뚜렸한 추상이란?")

#### Using LlamaIndex as a memory module

In [ ]:
# try using SummaryIndex!
from langchain.llms import OpenAI
from langchain.llms import OpenAIChat
from langchain.agents import initialize_agent

from llama_index import SummaryIndex
from llama_index.langchain_helpers.memory_wrapper import GPTIndexChatMemory

In [ ]:
index = SummaryIndex([])

In [ ]:
# set Logging to DEBUG for more detailed outputs
# NOTE: you can also use a conversational chain

memory = GPTIndexChatMemory(
    index=index,
    memory_key="chat_history",
    query_kwargs={"response_mode": "compact"},
    # return_source returns source nodes instead of querying index
    return_source=True,
    # return_messages returns context in message format
    return_messages=True,
)
# llm = OpenAIChat(temperature=0)
llm=OpenAI(temperature=0, openai_api_key='sk-')
agent_executor = initialize_agent(
    [], llm, agent="conversational-react-description", memory=memory
)

In [ ]:
agent_executor.run(input="hi, i am bob")

'Hi Bob, nice to meet you! How can I help you today?'

In [ ]:
# NOTE: the query now calls the SummaryIndex memory module.
agent_executor.run(input="what's my name?")

'What is your name?'